In [2]:
import modin.pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.recommendation import ALS as newALS
from sklearn.metrics import mean_squared_error
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
from sortedcontainers import SortedList, SortedDict
import time

In [3]:
num_features = 50
learning_rate = .02

In [4]:
np.random.seed(100)
initial_features = np.random.rand(num_features)

In [6]:
path = "../../Downloads/ml-1m"

def split_data(path):  
    df = pd.read_csv(f'{path}/ratings.dat', sep = "::", names=['user_id', 'movie_id', 'rating', 'timestamp']).sort_values('timestamp')
    start_ts = df['timestamp'].min()
    med_ts = df['timestamp'].quantile(.5)
    end_ts = df['timestamp'].max()
    train_df = df[df['timestamp'] <= med_ts]
    stream_df = df[df['timestamp'] > med_ts]
    seen_movies = set(train_df['movie_id'])
    print(len(seen_movies), len(set(stream_df['movie_id'])), len(stream_df))
    stream_df = stream_df.drop(stream_df[stream_df['movie_id'].map(lambda x: x not in seen_movies)].index)
    '''
    print(len(set(stream_df['movie_id'])), len(stream_df))
    seen_users = set(train_df['user_id'])
    num_stream_users = set(stream_df['user_id'])
    stream_df = stream_df.drop(stream_df[stream_df['user_id'].map(lambda x: x not in seen_users)].index)
    print(len(seen_users), len(num_stream_users), len(set(stream_df['user_id'])), len(stream_df))
    '''
    train_df.to_csv(f'{path}/train.csv', header=True, index = False)
    stream_df.to_csv(f'{path}/stream.csv', header=True, index = False)
    return start_ts, med_ts, end_ts

In [7]:
split_data(path)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

To request implementation, send an email to feature_requests@modin.org.


3551 3643 500104


(956703932, 973018006.0, 1046454590)

In [8]:
spark = SparkSession.builder.master('local').appName('als').getOrCreate()

22/03/25 22:55:04 WARN Utils: Your hostname, Amits-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.33 instead (on interface en0)
22/03/25 22:55:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/25 22:55:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
spark_df = spark.read.options(delimiter=",", inferSchema=True, header=True).csv(f'{path}/train.csv')

In [10]:
spark_df = spark_df.drop('timestamp')
spark_df.show()
new_model = newALS.train(ratings = spark_df.rdd, rank = num_features, iterations=15, lambda_=learning_rate)

+-------+--------+------+
|user_id|movie_id|rating|
+-------+--------+------+
|   6040|     858|     4|
|   6040|    2384|     4|
|   6040|     593|     5|
|   6040|    1961|     4|
|   6040|    2019|     5|
|   6040|    1419|     3|
|   6040|     213|     5|
|   6040|    3111|     5|
|   6040|     573|     4|
|   6040|    3505|     4|
|   6040|    1734|     2|
|   6040|    2503|     5|
|   6040|     919|     5|
|   6040|     912|     5|
|   6040|     527|     5|
|   6040|    1252|     5|
|   6040|     649|     5|
|   6040|     318|     4|
|   6040|    3289|     5|
|   6040|     759|     5|
+-------+--------+------+
only showing top 20 rows



22/03/25 22:55:24 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/03/25 22:55:24 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/03/25 22:55:24 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [11]:
user_features = new_model.userFeatures().toDF().toPandas()
user_features = user_features.rename(columns={'_1': 'id', '_2': 'features'})
product_features = new_model.productFeatures().toDF().toPandas()
product_features = product_features.rename(columns={'_1': 'id', '_2': 'features'})

22/03/25 22:59:28 WARN BlockManager: Task 49 already completed, not releasing lock for rdd_318_0


In [12]:
product_features.sort_values('id').to_csv(f'{path}/movie_features.csv', index = False)
user_features.sort_values('id').to_csv(f'{path}/user_features.csv', index = False)

In [13]:
def get_feature_dict(features):
    feature_dict = dict()
    for row in features.itertuples():
        feature_dict[int(row.id)] = np.array(row.features)
    return feature_dict

In [14]:
class FIFOScheduler:
    
    def __init__(self):
        self.queue = []
        self.name = "fifo"
        
    def push(self, row):
        self.queue.append(row)
    
    def pop(self):
        if len(self.queue) == 0:
            return None
        return self.queue.pop()

In [15]:
class FewestUpdateScheduler:
    
    def __init__(self):
        self.updated_list = SortedList(key = lambda x: x[1])
        self.row_dict = dict()
        self.name = "fewest"

    def push(self, row):
        user = row.user_id
        if user not in self.row_dict:
            self.updated_list.add([user, 0])
            self.row_dict[user] = [row]
        else:
            self.row_dict[user].append(row)
          
    def pop(self):
        user_data = self.updated_list.pop(0)
        user = user_data[0]
        row = self.row_dict[user].pop()
        if len(self.row_dict[user]) != 0:
            user_data[1] = user_data[1] + 1
            self.updated_list.add(user_data)
        #self.updated_list[0][1] = self.updated_list[0][1] + 1
        return row

In [50]:
class MaxPendingScheduler:
    
    def __init__(self):
        # items = [key, num_pending]
        self.updated_list = SortedList(key = lambda x: x[1])
        self.row_dict = dict()
        self.name = "pending"

    def push(self, row):
        user = row.user_id
        if user not in self.row_dict:
            self.updated_list.add([user, 1])
            self.row_dict[user] = [row]
        else:
            pending_updates = len(self.row_dict[user])
            self.updated_list.remove([user, pending_updates])
            self.updated_list.add([user, pending_updates + 1])
            self.row_dict[user].append(row)
        
    def pop(self):
        user = self.updated_list[-1][0]
        num_pending = len(self.row_dict[user])
        row = self.row_dict[user].pop()
        if num_pending == 1:
            self.updated_list.pop(-1)
            del self.row_dict[user]
        else:
            self.updated_list.remove([user, num_pending])
            self.updated_list.add([user, num_pending - 1])
        return row

In [17]:
class SGDUpdater:
    
    def __init__(self, user_features, movie_features, learning_rate, u_reg, m_reg, update_movies = True):
        self.user_dict = get_feature_dict(user_features)
        self.movie_dict = get_feature_dict(movie_features) 
        self.l = learning_rate
        self.u_reg = u_reg
        self.m_reg = m_reg
        self.update_movies = update_movies
        self.name = "sgd"
    
    def predict(self, row):
        movie_id = row.movie_id
        user_id = row.user_id
         
        if user_id not in self.user_dict:
            self.user_dict[user_id] = np.copy(initial_features)
        
        return self.user_dict[user_id].dot(self.movie_dict[movie_id].T)
    
    def update(self, rows):
        for row in rows:
            movie_id = row.movie_id
            user_id = row.user_id
            rating = row.rating

            prediction = self.predict(row)
            error = rating - prediction

            self.user_dict[user_id] = self.user_dict[user_id] + self.l * (error * self.movie_dict[movie_id] - self.u_reg  * self.user_dict[user_id])
            if self.update_movies:
                self.movie_dict[movie_id] = self.movie_dict[movie_id] + self.l * (error * self.user_dict[user_id] - self.m_reg * self.movie_dict[movie_id])

In [18]:
def get_feature_matrix(num_rows, features):
    matrix = np.zeros((num_rows, num_features))
    feature_dict = dict()
    for row in features.itertuples():
        feature_dict[int(row.id)] = np.array(row.features)
    for i in range(num_rows):
        if i in feature_dict:
            matrix[i] = feature_dict[i]
        else:
            matrix[i] = np.copy(initial_features)
    return matrix

In [19]:
class ALSUpdater:
    
    def __init__(self, user_features, movie_features, rating_matrix, confidence_matrix, max_user, max_movie, u_reg, m_reg):
        self.user_matrix = get_feature_matrix(max_user, user_features)
        self.movie_matrix = get_feature_matrix(max_movie, movie_features)
        self.rating_matrix = rating_matrix
        self.confidence_matrix = confidence_matrix
        self.u_reg = u_reg
        self.m_reg = m_reg
        self.name = "als"
        
    def predict(self, row):
        movie_id = row.movie_id
        user_id = row.user_id
        prediction = np.dot(self.movie_matrix[movie_id], self.user_matrix[user_id])
        return prediction
    
    def update(self, rows):
        for row in rows:
            movie_id = row.movie_id
            user_id = row.user_id
            rating = row.rating
            self.confidence_matrix[user_id][movie_id] = 1
            self.rating_matrix[user_id][movie_id] = rating
            user_confidence_row = self.confidence_matrix[user_id]
            left = np.dot(self.movie_matrix.T, np.dot(np.diag(user_confidence_row), self.movie_matrix)) + self.u_reg * np.eye(num_features)
            right = np.dot(self.movie_matrix.T, np.dot(np.diag(user_confidence_row), self.rating_matrix[user_id].T))
            print(self.movie_matrix.T.shape, np.dot(np.diag(user_confidence_row), self.rating_matrix[user_id].T).shape, left.shape, right.shape)
            self.user_matrix[user_id] = np.linalg.solve(
                    left,
                    right
                ).T       
        

In [20]:
stream_df = pd.read_csv(f'{path}/stream.csv')
data_to_send = dict()
for ts in stream_df['timestamp'].unique():
    curr_data = stream_df[stream_df['timestamp'] == ts]
    data_to_send[ts] = curr_data

(apply_func pid=72613) 


(apply_list_of_funcs pid=72616) 


(apply_list_of_funcs pid=72720) 
(apply_func pid=72616) 
(apply_list_of_funcs pid=72613) 
(apply_list_of_funcs pid=72719) 
(apply_list_of_funcs pid=72719) 


(apply_list_of_funcs pid=72721) 


(apply_func pid=72613) 
(apply_list_of_funcs pid=72618) 
(apply_list_of_funcs pid=72721) 


...

(apply_list_of_funcs pid=72618) 
(apply_list_of_funcs pid=72721) 


(apply_list_of_funcs pid=72613) 
(apply_list_of_funcs pid=72719) 


(apply_func pid=72718) 
(apply_func pid=72719) 


(apply_list_of_funcs pid=72616) 


(apply_list_of_funcs pid=72613) 
(apply_list_of_funcs pid=72615) 
(apply_list_of_funcs pid=72615) 


In [21]:
def simulate(scheduler, updater, stream, df, update_cadence, num_update):
    ts = df['timestamp'].min()
    end_ts = df['timestamp'].max()
    
    predictions = []
    next_feature_update_time = ts + update_cadence
    num_updates = 0
    num_processed = 0
    
    while ts <= end_ts:
        if ts in stream:
            curr_data = stream[ts]
            for row in curr_data.itertuples():
                scheduler.push(row)
                predictions.append(updater.predict(row))
            num_processed += len(curr_data)
        if ts >= next_feature_update_time:
            # how many records to process
            rows = []
            for i in range(num_update):
                row = scheduler.pop()
                if row:
                    rows.append(row)
            updater.update(rows)
            num_updates += len(rows)
            next_feature_update_time += update_cadence
        ts += 1
    return predictions

In [22]:
def run_simulations(scheduler_list, updater_list, stream, df, update_cadences, num_updates):
    experiments = []
    for scheduler in scheduler_list:
        for updater in updater_list:
            print("Scheduler")
            for update_cadence in update_cadences:
                for num_update in num_updates:
                    experiment_name = f'{scheduler.name}_{updater.name}_{update_cadence}_{num_update}_predictions'
                    start = time.time()
                    predictions = simulate(scheduler, updater, stream, df, update_cadence, num_update)
                    elapsed = time.time() - start
                    stream[experiment_name] = predictions
                    mse = mean_squared_error(stream['rating'], stream[experiment_name])
                    experiments.append([experiment_name, elapsed, mse])
    return stream, experiments

In [23]:
train_df = spark_df.toPandas()

In [24]:
max_user = train_df.max()[0]
if stream_df.max()[0] > max_user:
    max_user = stream_df.max()[0]
max_user += 1

max_movie = train_df.max()[1]
if stream_df.max()[1] > max_movie:
    max_movie = stream_df.max()[1]
max_movie += 1

max_user, max_movie

(6041, 3953)

In [26]:
def construct_matrices(df, max_user, max_movie):
    confidence_matrix = np.zeros((max_user, max_movie))
    ratings_matrix = np.copy(confidence_matrix)
    for row in df.itertuples():
        confidence_matrix[row.user_id][row.movie_id] = 1
        ratings_matrix[row.user_id][row.movie_id] = row.rating
    return ratings_matrix, confidence_matrix
    
rating_matrix, confidence_matrix = construct_matrices(train_df, max_user, max_movie)

In [52]:
scheduler_list = [MaxPendingScheduler(), FewestUpdateScheduler(), FIFOScheduler()]
sgd_updater = SGDUpdater(user_features, product_features, .02, .01, .01)
als_updater = ALSUpdater(user_features, product_features, rating_matrix, confidence_matrix, max_user, max_movie, 0, 0)
updater_list = [als_updater, sgd_updater]

In [53]:
stream_prediction_df, experiments = run_simulations(scheduler_list, updater_list, data_to_send, stream_df, [1000], [1])

Scheduler
(50, 3953) (3953,) (50, 50) (50,)


LinAlgError: Singular matrix